# Imports

In [ ]:
import glob
import itertools as it
import os
import re
from collections import namedtuple
from functools import partial
from pathlib import Path

import dask
import distributed
import h5py
import holoviews as hv
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import skimage.measure
import zarr
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from holoviews.operation.datashader import regrid
from tqdm.auto import tqdm, trange

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from paulssonlab.image_analysis import *
from paulssonlab.image_analysis.ui import display_image

In [ ]:
%load_ext pyinstrument

In [ ]:
hv.extension("bokeh")

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="06:00:00",
    memory="2GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(1)

In [ ]:
cluster.adapt(maximum=20)

In [ ]:
# filename = "/home/jqs1/scratch/jqs1/microscopy/230213/230213induction.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230215/230215induction.nd2" #v=7
# filename = "/home/jqs1/scratch/jqs1/microscopy/230326/230326promrbs.nd2" #v=8,t=10
# filename = "/home/jqs1/scratch/jqs1/microscopy/230404/230404_rbsprom.nd2"
# filename = workflow.SplitFilename(sorted(glob.glob("/home/jqs1/scratch/jqs1/microscopy/230619/230619_NAO745_repressilators_split.nd2*")))
filename = workflow.SplitFilename(
    sorted(
        glob.glob(
            "/home/jqs1/scratch/jqs1/microscopy/230707/230707_repressilators_restart.nd2.split.a*"
        )
    )
)
fish_filename = Path(filename).parent / "FISH/real_run"

In [ ]:
nd2 = workflow.get_nd2_reader(filename)

In [ ]:
nd2.metadata["channels"]

In [ ]:
nd2.sizes

In [ ]:
img = nd2.get_frame_2D(v=8, c=0, t=0)

In [ ]:
display_image(img, scale=0.9)

In [ ]:
display_image(img, scale=0.99)

# Radial distortion correction

In [ ]:
k1 = 8.947368421052635e-10
# k1 = 2e-8

In [ ]:
%%time
img_t = image.correct_radial_distortion(img, k1=k1)

In [ ]:
%%time
diag = util.tree()
trenches, info = trench_detection.find_trenches(
    img_t, width_to_pitch_ratio=1.4 / 3.5, join_info=False, diagnostics=diag
)

In [ ]:
info

In [ ]:
diag["bboxes"]

# Radial distortion optimization

In [ ]:
k1s = np.linspace(-1e-9, 2e-9, 20)
res = []
for k1 in tqdm(k1s):
    img_corrected = image.correct_radial_distortion(img, k1=k1)
    h, theta, rho = trench_detection.hough.hough_line_intensity(
        img_corrected, theta=np.linspace(-np.pi / 50, np.pi / 50, 400)
    )
    smooth = 4
    diff_h = np.diff(h.astype(np.int_), axis=1)  # TODO: is diff necessary??
    diff_h_std = diff_h.std(axis=0)  # / diff_h.max(axis=0)
    if smooth:
        diff_h_std_smoothed = scipy.ndimage.gaussian_filter1d(diff_h_std, smooth)
    else:
        diff_h_std_smoothed = diff_h_std
    theta_idx = diff_h_std_smoothed.argmax()
    diff_h_std_max = diff_h_std_smoothed[theta_idx]
    angle = theta[theta_idx]
    res.append(
        dict(
            k1=k1,
            h=h,
            diff_h=diff_h,
            diff_h_std=diff_h_std,
            diff_h_std_smoothed=diff_h_std_smoothed,
            angle=angle,
            theta_idx=theta_idx,
            img_corrected=img_corrected,
        )
    )

In [ ]:
k1

In [ ]:
slice_ = slice(150, 230)

In [ ]:
plt.figure(figsize=(20, 20))
for idx in range(len(res)):
    plt.plot(res[idx]["diff_h_std"][slice_], label=idx)
plt.legend()
# plt.plot(res[12]["diff_h_std"], lw=4)

In [ ]:
plt.figure(figsize=(20, 20))
idx = 12
plt.plot(res[idx - 2]["diff_h_std"][slice_])
plt.plot(res[idx]["diff_h_std"][slice_], lw=4)
plt.plot(res[idx + 2]["diff_h_std"][slice_])

In [ ]:
res[12]["k1"]

In [ ]:
display_image(res[-3]["img_corrected"], scale=True)

# FISH correction

In [ ]:
# filename = "/home/jqs1/scratch/jqs1/microscopy/230213/230213induction.nd2"
# filename = "/home/jqs1/scratch/jqs1/microscopy/230215/230215induction.nd2" #v=7
# filename = "/home/jqs1/scratch/jqs1/microscopy/230326/230326promrbs.nd2" #v=8,t=10
filename = "/home/jqs1/scratch/jqs1/microscopy/230404/230404_rbsprom.nd2"
fish_filename = Path(filename).parent / "FISH/real_run"

In [ ]:
k1 = 8.947368421052635e-10

In [ ]:
def calibrate_image(img, k1=0):
    img = skimage.img_as_float32(img)
    img = image.correct_radial_distortion(img, k1=k1)
    return img

In [ ]:
%%time
delayed = util.get_delayed(True)
fish_frames = {}
fish_crops = {}
fish_channels = set()
fish_timepoints = set()
for msg in readers.send_eaton_fish(
    fish_filename,
    r"fov=(?P<v>\d+)_config=(?P<c>\w+)_t=(?P<t>\d+)",
    slices=dict(t=None, v=[8]),
    delayed=delayed,
):
    # print(msg["metadata"],msg["image"].shape)
    fish_img = msg["image"]
    # fish_img_corrected = delayed(calibrate_image)(fish_img, k1=k1)
    fish_img_corrected = delayed(skimage.img_as_float32)(fish_img)
    t = msg["metadata"]["t"]
    channel = msg["metadata"]["channel"]
    fish_channels.add(channel)
    fish_timepoints.add(t)
    fish_frames[(t, channel)] = fish_img_corrected
fish_channels = list(sorted(fish_channels))
fish_timepoints = list(sorted(fish_timepoints))

In [ ]:
fish_frames0 = dask.compute(fish_frames)[0]

In [ ]:
%%time
stacks = {}
for channel in fish_channels:
    for timepoint_idx, timepoint in enumerate(fish_timepoints):
        img = fish_frames0[(timepoint, channel)]
        if channel not in stacks:
            stacks[channel] = np.full((len(fish_timepoints), *img.shape), np.nan)
        stacks[channel][timepoint_idx, :, :] = img

In [ ]:
stacks["GFP"].shape

In [ ]:
stacks["GFP"][3:9].max(axis=0)

In [ ]:
stacks["GFP"][3:9].max(axis=0)

In [ ]:
display_image(stacks["GFP"][3:9].max(axis=0), scale=0.99)

In [ ]:
display_image(stacks["GFP"][3:9].min(axis=0), scale=0.99)

In [ ]:
display_image(
    stacks["GFP"][3:9].max(axis=0) - stacks["GFP"][3:9].min(axis=0), scale=0.99
)

In [ ]:
display_image(stacks["GFP"][:9].max(axis=0) - stacks["GFP"][:9].min(axis=0), scale=0.99)

In [ ]:
info